In [1]:
import json
import sys, os
sys.path.append(os.path.join('build/python'))
import suggestr

In [2]:
def open_dictionary(path):
    path = os.path.expanduser(path)
    fs = os.path.getsize(path)
    return suggestr.load_dictionary(path, 0, fs, 0)

In [3]:
D, sess = open_dictionary("dicts/main_de.dict")

In [4]:
D2, sess2 = open_dictionary("dicts/chromium_en_US.dict")

In [5]:
D2.get_word_prop("news")

In [6]:
w = D.get_word_prop("Absperrbauwerk")

In [7]:
w.get_unigram_prop()

In [8]:
w.get_bigram_props()

[]

In [9]:
up = w.get_unigram_prop()

In [10]:
def print_props(word, D):
    w = D.get_word_prop(word)
    up = w.get_unigram_prop()
    bg = w.get_bigram_props()
    print("\n=== {} ===\n".format(word))
    if len(bg):
        print("BIGRAM: {}".format(len(bg)))
        for b in bg:
            print("  - {}".format(b.get_target()))
    print("Prob: {}".format(up.get_probability()))
    print("Shortcuts: {}".format(up.has_shortcuts()))
    print("Lvl: {}".format(up.get_level()))
    print("Is Not A Word: {}".format(up.is_not_a_word()))
    print("Blacklist: {}".format(up.is_blacklisted()))

In [11]:
print_props('Absperrbauwerk', D)


=== Absperrbauwerk ===

Prob: 70
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False


In [12]:
key_layout = suggestr.layout_to_keys('./layouts/german.yaml')

['q', 'w', 'e', 'r', 't', 'z', 'u', 'i', 'o', 'p']
['a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l']
['Shift_L', 'y', 'x', 'c', 'v', 'b', 'n', 'm', 'BackSpace']
['show_numbers', 'show_dechars', 'preferences', 'space', ',', 'period', 'Return']


In [13]:
a = [1,2,3,4]
print(a[2])
print(a[:2])

3
[1, 2]


In [16]:
up = suggestr.UnigramProperty(representsBeginningOfSentence=False, isNotAWord=False, isBlacklisted=False, 
                              probability=100, timestamp=0, level=0, count=1, shortcuts=[])

In [17]:
import re

def clean(word):
#     word = word.replace('-', '--')
    word = re.sub('[^0-9a-zA-Z\'\s\-]+', '', word)
    return word

In [18]:
with open('/home/wolfv/Downloads/all_movies.json') as fi:
    J = json.load(fi)

all_words = set()
wc = {}

for el in J[:]:
    label = el['itemLabel']
    label = clean(label)
#     spw = label.split()
    spw = [label]
    for w in spw:
        if wc.get(w):
            wc[w] += 1
        else:
            wc[w] = 1

In [19]:
max_wc = -1
for k, v in wc.items():
    max_wc = max(v, max_wc)

In [20]:
wc_list = wc.items()

In [21]:
print(sorted(wc_list, key=lambda x1: x1[1])[-25:])

[('Finders Keepers', 5), ('Behind the Mask', 5), ('The Spoilers', 5), ('Jack and the Beanstalk', 5), ('Stage Struck', 5), ('Outcast', 5), ('The Escape', 5), ('Reunion', 5), ('Seven Keys to Baldpate', 5), ('The Mysterious Rider', 5), ('Home of the Brave', 6), ('The Kiss', 6), ('Camille', 6), ('Exposed', 6), ('Mercy', 6), ('The Trap', 6), ("Uncle Tom's Cabin", 6), ('Driftwood', 6), ('Evidence', 6), ('The Outsider', 6), ('Jane Eyre', 7), ('Treasure Island', 7), ('Resurrection', 7), ('Body and Soul', 7), ('The Three Musketeers', 9)]


In [22]:
base_score = 300
def score(word):
    count = wc[word]
    if count > 250:
        return 251
    else:
        return int(float(count) / float(base_score) * 250) + 1

In [23]:
def to_unigram(w):
    return w.replace(' ', '-')

In [25]:
flushcount = 0
ND, nsess = suggestr.make_dictionary("de_DE")
for el in J[:1000]:
    label = el['itemLabel']
    label = clean(label)
#     spw = label.split()
    spw = [label]
    for w in spw:
        if w not in all_words:
            all_words.add(w)
            print(w, score(w))
            # if one uses the default (255) unigram prob here, bigrams get discarded!
#             ND.add_unigram(w.replace(' ', ''), suggestr.UnigramProperty(
#                 False, False, False, score(w), 0, 0, 1, []))
            word = to_unigram(w)
            ND.add_unigram(word, suggestr.UnigramProperty(
                False, False, False, 100, 0, 0, 1, []))
#     pwi = [suggestr.PrevWordInfo(pw, False) for pw in spw]
#     flushcount += 1
#     if flushcount > 200:
#         ND.flush_with_gc('./testdict/')
#         flushcount = 0
#     for i in range(1, len(pwi)):
#         bp = suggestr.BigramProperty(spw[i], 240, 0, 0, 1)
#         ND.add_ngram(pwi[:i], bp)

We Live in Public 1
Kick-Ass 1
12 Angry Men 1
Gone with the Wind 1
Rough Night in Jericho 1
Alien Raiders 1
The Manchurian Candidate 2
Cleopatra 4
Wanted 2
Delirious 2
Matchstick Men 1
The Oranges 1
Skyfall 1
V for Vendetta 1
Pirates II Stagnetti's Revenge 1
Gotcha 1
Special Effects Anything Can Happen 1
Dances with Wolves 1
Signs 1
Fear and Loathing in Las Vegas 1
Africa Screams 1
Abbott and Costello in the Foreign Legion 1
Godzilla 2
Alligator II - The Mutation 1
Visions of Light 1
The Last Airbender 1
Eraserhead 1
ET the Extra-Terrestrial 1
Swing Vote 1
Hackers 1
The Day After Tomorrow 1
Oxygen 1
The Sisterhood of the Traveling Pants 2 1
Thursday 1
Star Wars Episode IV A New Hope 1
Fellini A Director's Notebook 1
Birthday Girl 1
National Lampoon's Van Wilder 1
The Magnificent Seven 2
GoldenEye 1
Paradise Alley 2
The Ninth Gate 1
Life or Something Like It 1
Die Another Day 1
The Double Man 1
Tell Me a Riddle 1
The Seventh Victim 1
Assassination Games 1
More Than a Miracle 1
The Compa

Halloween Resurrection 1
Reunion in Vienna 1
PT 109 1
The Forbidden Kingdom 1
The Principal 1
Nim's Island 1
There's Something About Mary 1
Prisoner of Paradise 1
To Kill a Mockingbird 1
Enemy of the State 1
Bamako 1
The Notorious Bettie Page 1
The Girl Who Had Everything 1
Shotgun Stories 1
Marie and Bruce 1
The Invention of Lying 1
Liberty Heights 1
Seabiscuit 1
Enough 2
6 Guns 1
Indiana Jones and the Temple of Doom 1
About Schmidt 1
Bringing Up Baby 1
Cry-Baby 1
Kalifornia 1
Canadian Bacon 1
Ben-Hur 2
Step Up 1
Black Swan 1
Never Say Never Again 1
15 Minutes 1
Dead Mary 1
Green Street 2 Stand Your Ground 1
National Velvet 1
Death Warrant 1
Drawing Restraint 9 1
The Lost City 3
The Four Days of Naples 1
Just Visiting 1
The Singing Fool 1
Tarzan and the Mermaids 1
Paris Underground 1
Letters from a Killer 1
Red Corner 1
The Best Man 3
Friday the 13th A New Beginning 1
Noah 1
Ceiling Zero 1
Stateside 1
Waking the Dead 1
Gangs of the Dead 1
Nights in Rodanthe 1
The Miracle Worker 1
Me a

In [27]:
def get_with_context(context, D, sess):
    if context[-1].isspace():
        [print(pw) for pw in context.split()]
        prev_words = [suggestr.PrevWordInfo(pw.strip(), False) for pw in context.split()]
        x = suggestr.get_suggestion("", prev_words[::-1], D, sess, key_layout)
        print(x)
    else:
        s = context.split()
#         prev_words = [suggestr.PrevWordInfo(pw, False) for pw in s[:-1]] if len(s) else []
#         x = suggestr.get_suggestion(s[-1], prev_words[::-1], D, sess)
#         context = context.replace(' ', 'ä')
        print(to_unigram(context))
        x = suggestr.get_suggestion(to_unigram(context), [], D, sess, key_layout)
        print(x)

In [28]:
# print([x['itemLabel'] for x in J[:1000]])

In [31]:
get_with_context('ypo', ND, nsess)

ypo
['Zoo', 'Apollo-13', 'Apollo-18', 'Zoom', 'Zoolander', 'Zoo-in-Budapest', 'Split-Second', "Sophie's-Choice", 'A-Lot-Like-Love', 'Alive', 'Alien', 'Aliens', 'Alien-Raiders', 'Aliens-vs-Avatars', 'Alien-vs-Predator', 'Soldier-Boyz', 'All-Good-Things']


In [25]:
get_with_context('meninlback i', ND, nsess)

meninlback-i
['Men-in-Black-II', 'Men-in-Black', 'Onegin', 'Felon', 'Argo', 'Noah']


In [26]:
next_t = 0
for i in range(100):
    next_t, word = ND.get_next_word_and_token(next_t)
    print_props(word, ND)


=== We-Live-in-Public ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== West-Side-Story ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Western-Union ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Wanted ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Warm-Bodies ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== War-Horse ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Walk-the-Line ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Wall-Street ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Watchmen ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Waterworld ===

Prob: 100
Shortcuts: False
Lvl: 0
Is Not A Word: False
Blacklist: False

=== Way-Out-West ===

Prob: 100
Shortcuts: False
Lvl: 0
